# Electroencephalograpic classification of human non-REM sleep stages 
- Electroencephalography (**EEG**) is a widely used neurophysiological method to measure brain electrical activity from the scalp surface.
- The measured voltages (potential differences) reflect the sum of post-synaptic potentials (both EPSP and IPSP) across millions of neurons
- EEG is commonly used to:
  - determine sleep stages
  - measure increased cortical excitability in epilepsy (so-called sharp waves, spikes, seizure patterns)
  - objectively measure _disordered_ brain activity in e.g. encephalopathies (infections, metabolic disorders, intoxications, ...)

**Advantages**:
- The signal reflects actual neuronal electrical activity; other methods such as functional Magnetic Resonance Imaging (fMRI) or functional near-infrared spectroscopy (fNIRS) measure hemodynamic changes (increased blood flow) as an _indirect_ measure of increased neuronal signalling
- EEG is a relatively cheap technology (electrodes, amplifier, visualization software); roughly thousands of \\$ compared to millions of \\$ for MRI
- High temporal resolution. EEG is usually sampled at several hundred Hertz (1 Hz = 1 sample /second), but can easily be extended to the kHz range if needed
- EEG is risk-free. Some structural imaging methods use X-rays (computed tomography, CT), and some functional imaging methods use radioactive markers (positron emission tomography/PET, single-photon emission computerized tomography/SPECT)

**Disadvantages**:
- Low spatial resolution: brain electric currents disperse as they travel through brain tissue, the cerebro-spinal fluid, bone and skin, before they reach the scalp electrodes. This phenomenon is called _volume conduction_ and explains why EEG has low spatial resolution. Roughly, it can be said that EEG allows localizazion of activity on the lobar level (i.e. something happens in the left frontal lobe, right temporal lobe, etc.)
- Artefacts: EEG measurements are easily influenced by non-neural sources, for example: muscle activity, eye movements, electrode movements, sweating and other changes in skin conductivity

<img src="img/brain_lobes_wiki.png" alt="brain lobes" style="display: block; margin-left: auto; margin-right: auto; width: 75%;"/>

To understand the signals we will be analyzing in this practical, you have to understand the **EEG cap layout**:  
Electrode names: 
- **_Even_** numbers are over the **_right_** hemisphere
- **_Odd_** numbers are over the **_left_** hemisphere,
- The letter 'z' (e.g. Fz, Cz, Pz) indicates an electrode over the central line (nasion to inion line)
- The letters **F, T, P, O** indicate the nearest brain lobe, **Fp** denotes fronto-polar, and **C** indicates the region around the central sulcus; the **A** electrodes are attached near the ears ('auricular') and are assumed to record _no_ relevant brain activity

**Knowledge check**:
- The precentral gyrus belongs to the _____ lobe, and its function is _____.
- The postcentral gyrus belongs to the _____ lobe, and its function is _____.
- Both pre- and post-central gyrus activity projects on central electrodes. Activity in the left precentral gyrus would mainly project on electrode ___, activity in the right postcentral gyrus mainly projects on electrode ___.
- Visual processing is localized in the _____ lobe, activity of the right visual cortex would project on electrode ___, left visual cortex activity would mainly project on electrode ___. 

<img src="img/21_electrodes_of_International_10-20_system_for_EEG.svg.png" alt="10-20 system" style="display: block; margin-left: auto; margin-right: auto; width: 50%;"/>

**_Fun fact_**: The EEG research community celebrates **100 years** of human EEG in 2024. Earlier measurements were conducted by the British physician and physiologist Richard Caton around 1875, but the technique was finally established and systematically investigated by Hans Berger in Jena, Germany, who did his first measurements on his children in 1924. 

<img src="img/HansBerger_EEG.png" alt="Hans Berger" style="display: block; margin-left: auto; margin-right: auto; width: 40%;"/>


# Learn basic EEG sleep classification in 10 minutes!
- Every time you execute the code cell below, two randomly selected EEG segments (length: 8 seconds) will be shown. The dataset contains data from n=14 healthy adult volunteers. The two traces will always be taken from the same subject. When you run the code cell again, another subject will be chosen randomly.  
  - One channel is taken from non-REM sleep stage **W** (W = wakefulness), the other from sleep state **N1** (N1: light sleep). These labels were set by a human scorer using the rules established by the AASM (American Academy of Sleep Medicine) in 2007, [https://aasm.org/clinical-resources/scoring-manual](https://aasm.org/clinical-resources/scoring-manual/).
  - The default EEG channel is O1. Where is this electrode located? You can vary the display EEG channel by editing the function call `channel = 'O1'` in the function call to `channel = 'C4'` or any other channel from the set: Fp1, Fp2, F3, F4, C3, C4, P3, P4, O1, O2, F7, F8, T7, T8, P7, P8, Fz, Cz, Pz, AFz, AF3, AF4, FC3, FC4, FT9, FT10, TP9, TP10, CP5, CP6 (don't forget the quotes!).
- The bottom panel shows the frequency spectra (power spectral density) of the EEG datasets from which the 8 sec traces were taken, using the same color code.

**TASKS**
- Try you identify which EEG trace is from wakefulness (W) and which one is N1 sleep?
  1. During **wakefulness**, healthy adults show **alpha oscillations (8-12 Hz)** over **posterior brain regions**.
  2. The **AASM Manual** for the scoring of sleep (2007) asks the human scorer to label those EEG epoch as **N1 (light sleep) "_if alpha rhythm is replaced by low amplitude, mixed frequency activity for more than 50% of the epoch_"**. The rules should be applied to 30 second EEG pages but we will try on short 8 sec. epochs.
  3. Report whether all epochs are equally easy or difficult to score. Describe difficulties that occur with visual scoring.
  4. Test the general approach to focus on posterior (occipital, parietal) electrodes to detect W/N1 changes.
  5. Observe and describe the peak frequencies of alpha oscillations (8-12 Hz range).
     - Do all subjects have the same peak alpha frequency?
     - Given that alpha oscillations reflect signals propagating in thalamo-cortical circuits, describe anatomical/physiological factors that could affect individual alpha frequencies.

## <mark> Below follows the first Python code cell. You must run this code to import all necessary functions. </mark>
You can run this cell with `Ctrl+Enter`, or `Shift+Enter`,  or by pressing the Play/Run button in the top menu.

In [ ]:
from ml_nrem import *

In [ ]:
# run this code cell several times to inspect random 10 sec EEG snippets
fig = show_random_data(channel = 'O1')
plt.show()

## Machine Learning Classifiers - a very short introduction

We will now test whether a non-human classifier can learn sleep stage classification.  
Numerous classification methods based on a wide range of mathematical-statistical techniques are usually summarized under the name of **Machine Learning (ML)**.
- Machine Learning forms a subset of what is understood as **Artificial Intelligence (AI)**.
- A widely used programming framework for Machine Learning is the Python library [scikit-learn](https://scikit-learn.org/stable/).
- **Python** has been the most popular programming language for years and is one of the top skills requested by employers in the tech world:
[Top Programming Languages 2024](https://spectrum.ieee.org/top-programming-languages-2024)
- In this practical, we will employ a technique called **Random Forest Classification**
- As any good forest, a random forest is made of **trees**. In this context, a tree in the forest is a **decision tree**. You are probably using decision trees in your daily life without calling them so.
- Decision Tree example: before you leave the house in the morning, you might check the outside temperature and the chance of rain. This data goes into a decision tree that determines your clothes for the day.

<img src="img/decision_tree_weather.png" alt="Decision Tree weather" style="display: block; margin-left: auto; margin-right: auto; width: 40%;"/>

<!--
https://miro.medium.com/v2/resize:fit:640/format:webp/1*i0o8mjFfCn-uD79-F1Cqkw.png
-->

## A Random Forest for sleep stage classification

1. The full **Random Forest algorithm** is more complicated than the decision tree described above. It uses a stochastic (randomized) approach, where different trees in the forest receive a random subset of the features that we use for classification.
2. Our Machine Learning algorithm belongs is a so-called **supervised learning** method. Supervision happens in the **training phase** during which the algorithm receives a large number of (features, target variables) pairs. They provide the associations to be learned:
- **Features** are the input variables that the Random Forest will receive for classification. In our simplified example above, the features would be the data pair (temperature, rain).
- **Target variables** contain the desired classification **output**.
3. In our case, we will try to classify sleep from **spectral EEG properties**, similar to what you did in the first part of this practical. You may have observed that there was higher alpha power over posterior electrodes during wakefulness, and lower alpha power during N1. Your brain then made a very simple decision tree that decided to label the EEG epoch with higher alpha power 'W' and the epoch with lower alpha power 'N1'.
4. Our **features** are a set of 120 values: 4 spectral power values (one for each of the 4 frequency bands: delta, theta, alpha, beta) x 30 EEG electrodes. They will be computed and averaged over a 10 second EEG epoch. One minute of EEG therefore gives us 6 examples.
5. For each feature set, we have one **target variable** which represents the actual sleep stage. For algorithms, it is easier to use numbers so we encode them as W: 0, N1: 1, N2: 2, N3: 3.
6. During the **training phase**, the algorithm will learn to make the decision trees in the forest better. Since we provide both, features and the correct sleep stage (**target variable**), the algorithm has a chance to minimize the number of incorrectly classified epochs. The details of the algorithm go beyond this practical but can be found here: [Breiman, L (2001) Random forests. Machine Learning 45(1):5–32](https://link.springer.com/article/10.1023/A:1010933404324), the Python source code can be found here: [scikit-learn Random Forest code](https://github.com/scikit-learn/scikit-learn/blob/f5aac2173/sklearn/ensemble/_forest.py#L1170).
7. During the **test phase**, the trained classifier is applied to EEG epochs it **_has_** not seen during training. This is used to avoid **overfitting**. An algorithm can learn too much about the specific features of a training data set, at the cost of poor generalization. The optimal amount of learning cannot be predicted and has to be determined empirically, using the **training/test split** approach. A common choice is to use 80% of the data for training, and 20% for testing. This is called a 80/20 split in the literature.

### Select the EEG frequency bands for classification
- Set these variables to `True` to use the corresponding frequency band for classification, and to `False` to exclude it.

In [ ]:
# select frequency bands: True, False
delta = True
theta = True
alpha = True
beta = True

### Compute features and targets
The next code cell computes the spectral features, orders them in a 2D matrix (features: `X`) and makes the list of correct sleep stage labels (targets: `y`)

In [ ]:
print("Computing features, be patient...")
X, y, feature_names = make_features(delta, theta, alpha, beta)
print("Features and targets ready to be used.")

Next, visualize the matrix `X` to get an idea of the features that the Random Forest will learn to classify.  
To facilitate visualization, the matrix `X` is transposed, i.e. each column is a set of 120 features, representing 10 seconds of EEG. The sleep stages W, N1, N2, N3 appear in order, but could be presented in any order.  
Below, the full list of feature names is given.  

**Questions**:
1. What do the higher values of the features 60-70 (approximately) in stage W represent?
2. Estimate how many minutes of EEG we are using for classification?
3. How do the feature values 0-30 change from W to N3? Does this align with what you learned in the lecture?

In [ ]:
# plot features
bounds = 1+np.where(np.diff(y))[0] # boundaries between sleep states
#print(bounds)
midpoints = []
for i in range(len(bounds)):
    if i==0:
        midpoints.append(int(bounds[i]/2))
    else:
        midpoints.append(bounds[i-1]+int((bounds[i]-bounds[i-1])/2))
midpoints.append(bounds[-1]+int((len(y)-bounds[-1])/2))
#print(midpoints)
stages = ['W', 'N1', 'N2', 'N3']
plt.figure(figsize=(18,9))
plt.imshow(X.T, cmap=plt.cm.bwr)
ax = plt.gca()
for b in bounds:
    ax.axvline(b, color='k')
for i, m in enumerate(midpoints):
    #ax.axvline(m, color='g')
    ax.text(x=m, y=-3, s=stages[i])
plt.title("EEG spectral features for classification")
plt.tight_layout()
plt.show()
for i, f in enumerate(feature_names):
    print(f"{i:d}: {f:s}")

In [ ]:
#print(feature_names)

### 80/20 split and setup the classifier
We will split the input (features `X`) and output (targets `y`) variables into two data sets, 80% training data and 20% test data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)
print(f"X_train: {X_train.shape[0]:d} samples, {X_train.shape[1]:d} features")
print(f"y_train: {y_train.shape[0]:d} samples")
print(f"X_test: {X_test.shape[0]:d} samples, {X_test.shape[1]:d} features")
print(f"y_test: {y_test.shape[0]:d} samples")

### Hyperparameter optimization
The Random Forest has so-called **hyperparameters**. They are properties that characterize the decision trees in the forest and we don't know what the best values for a given classification task will be.  
We will therefore try different values and perform a **cross-validation** procedure. During cross-validation, only a part of the training data is used to train the classifier, which is then tested on the remaining data. This is done for 10 different splits. **Note:** this approach is identical to the training/test split explained above, but is only applied to the training data. The test data is still hidden from the classifiers and will only be used for the final assessment of the optimized Random Forest classifier.

In [ ]:
test_params = { 
    'max_features': [5, 15, 25], 
}

# Here are a few more parameters that you could try to optimize, either one or several at a time
# to do this, you can comment the test_params definition above, and uncomment the one below; 
# lines of code starting with # are not executed by the Python interpreter
#test_params = { 
#    'max_features': [5, 15, 25], 
#    'n_estimators': [100, 200, 300],
#}

fixed_params = {
    'max_depth': 10,   
    'n_estimators': 100,
    'n_jobs': -1,
    'random_state': 42,
}

tuning = GridSearchCV(
    cv = 10,
    estimator = RandomForestClassifier(**fixed_params), 
    n_jobs = -1, 
    param_grid = test_params, 
    scoring = 'accuracy',
)
print("Searching for optimal classifier parameters, be patient...")
tuning.fit(X_train, y_train) # this can take some time...
print("Optimal parameters found: ", tuning.best_params_)
print(f"Best score obtained on training data: {tuning.best_score_:.3f}")
clf_opt = tuning.best_estimator_

- Is there anything concerning about the optimal classifier parameters found?
- How would you address this problem?

### Evaluate the optimized classifier on the test data set
Remember that the classifier has been optimized on the training data set and has not yet seen the test data.  
It will therefore be interesting to see how well the classifier does on data it has never seen, let's do this now:

In [ ]:
test_score = clf_opt.score(X_test, y_test)
print("Accuracy on test data: ", test_score)

**Questions:**
1. Compare the classification accuracy from the optimization procedure with the score obtained on test data?
2. Compare with other groups in your class, is there a systematic difference? Explain the results!

In [ ]:
# optional: save the model to disk
f_clf_opt = f"./RFC_opt.pkl"
print(f"Optimized classifier saved as: {f_clf_opt:s}")
with open(f_clf_opt, 'wb') as fp:
    pickle.dump(clf_opt, fp)

### Feature importance
We would now like to find out which spectral EEG features the classifier has actually used for classification. Fortunately, the algorithm has stored the feature importance and we can output the ordered list, from high to low importance. The total sum of importances will add up to 100%.  
For now, we will only look at the 20 most important features.

**Questions:**
1. Which electrode positions seem to be important to the classifier?
2. Which frequency bands seem to be important to the classifier?
3. Which electrode/frequency combinations are important?
4. Compare your results with those of other groups.
5. How did occipital alpha power score? We used these values in the first task (visual W-N1 scoring)

In [ ]:
idx_sort = np.argsort(clf_opt.feature_importances_)[::-1]
feature_importances_sorted = np.array(clf_opt.feature_importances_)[idx_sort]
feature_names_sorted = np.array(feature_names)[idx_sort]
feat_max = 20
for i, fimp in enumerate(feature_importances_sorted[:feat_max]):
    print(f"{i:d}, {feature_names_sorted[i]:s}: {100*fimp:.1f}%")

## Confusion matrix

In [ ]:
y_predicted = clf_opt.predict(X_test)
conf_mat = confusion_matrix(y_test, y_predicted)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=sleep_stages)
disp.plot()
plt.title(f"Confusion matrix\nRandom Forest Classification of sleep_stages (spectral features)")
plt.show()

**Questions:**
1. Give a verbal explanation of the confusion matrix, what does it tell you?
2. Why is the term _confusion_ used?
3. Which sleep stages does the Random Forest Classifier confuse?
4. Which sleep stage is the least likely to be confused with another sleep stage?
5. Which sleep stage is the most likely to be confused with another sleep stage?
6. Does the classifier agree with the human definition of which sleep stages are adjacent to each other?

In [ ]:
acc = clf_opt.score(X_test, y_test)
y_predicted = clf_opt.predict(X_test)
class_report = classification_report(y_test, y_predicted)
print(class_report)

## Plot some decision trees

In [ ]:
from sklearn import tree
tree_idx = 3
tree.plot_tree(clf_opt.estimators_[tree_idx])
plt.savefig(f"decision_tree{tree_idx:d}.png", dpi=1200)
plt.show()

## Competition!
- You have now trained a classifier using a set of EEG frequency bands (delta, theta, alpha, beta), and using optimized hyperparameters
- Now, modify some of the parameters (frequency bands included for classification, hyperparameters) and record your best perfomance on the **test data set**.
- Compare your best accuracy values with other groups in the prac and determine the winner!


## Additional exercise: cross-validate the classifier against randomly shuffled target labels
To be sure that the classifier is performing better than chance level, let's compare the average classification accuracy across ten different train/test splits (=cross-validation) for:
- the trained classifier with **_true_** target labels against
- the trained classifier with **_randomly shuffled_** target labels
Only if the classifier has learned 'meaningful' data properties, we will be able to observe a better perfomance on true labels.
 

In [ ]:
# F1-score cross-validation
n_cv = 10

print(f"\n[+] Cross-validation (N={n_cv:d}) on TRUE labels (wait...)")
folds = StratifiedShuffleSplit(n_splits = n_cv, train_size = 0.8)
scores = []
for idx_train, idx_test in folds.split(X, y):
    X_train, y_train, X_test, y_test = X[idx_train], y[idx_train], X[idx_test], y[idx_test]
    #clf_opt.fit(X_train, y_train)
    y_pred = clf_opt.predict(X_test) # [:, 1]
    f1 = f1_score(y_test, y_pred, average = None, labels = [1])[0]
    scores.append(f1)
print(f"F1-scores: mean={np.mean(scores):.2f}, std={np.std(scores):.2f}")

print(f"\n[+] Cross-validation (N={n_cv:d}) on SHUFFLED labels (wait...)")
y_shuffled = np.random.permutation(y) # test against shuffled labels
folds = StratifiedShuffleSplit(n_splits = n_cv, train_size = 0.8)
scores_shuffled = []
for idx_train, idx_test in folds.split(X, y_shuffled):
    X_train, y_train, X_test, y_test = X[idx_train], y_shuffled[idx_train], X[idx_test], y_shuffled[idx_test]
    #clf_opt.fit(X_train, y_train)
    y_pred = clf_opt.predict(X_test) # [:, 1]
    f1 = f1_score(y_test, y_pred, average = None, labels = [1])[0]
    scores_shuffled.append(f1)
print((f"F1-scores: mean={np.mean(scores_shuffled):.2f}, std={np.std(scores_shuffled):.2f}"))

In [ ]:
from scipy.stats import mannwhitneyu
t, p_mw = mannwhitneyu(scores, scores_shuffled)
print(f"\n[+] Mann-Whitney U test: p = {p_mw:.4f}")
alpha = 0.05
if (np.mean(scores) > np.mean(scores_shuffled) and p_mw < alpha):
    print("Classifier performance IS statistically significant.")
else:
    print("Classifier performance IS NOT statistically significant.")

# Conclusions and summary
1. We have trained Random Forest classifiers in a supervised manner to recognize non-REM sleep stages from EEG data.
2. We decided to use spectral power at each EEG electrode as the features. There are many other possible features that could be computed from EEG data and that could be used for classifier training.
3. Random Forest classifiers were able to learn non-REM sleep classification.
4. The feature importance assigned by the Machine Learning algorithm is not that different from the features that humans use.
5. The Machine Learning classifier makes mistakes. The confusion occurs for adjacent sleep stages. For examples, W is sometimes misclassified as N1, but not as N2 or N3.
6. When you re-visit this practical, make sure you understand the following concepts:
   - What is supervised classification?
   - What does 'features' mean in the context of Machine Learning?
   - Why do we split the data into training and test data sets?
   - What are hyperparameters?
   - Explain the concept of cross-validation.